In [ ]:
pip install wrds

     |████████████████████████████████| 3.0 MB 6.0 MB/s 


In [ ]:
import wrds
import pandas as pd
conn = wrds.Connection()
conn.list_libraries().sort()
type(conn.list_libraries())

Enter your WRDS username [root]:mhpour
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


list

In [ ]:
crsp = conn.raw_sql("""
                      select *
                      from crsp.dsf as a
                      left join crsp.dsenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '17/06/2005' and '26/08/2005'
                      and ABS(a.prc)*a.shrout>10000000
                      """, date_cols=['date'])

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 22))



DataError: ignored

In [ ]:
crsp1 = conn.raw_sql("""
                      select *
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/2020' and '12/31/2021'
                      and ABS(a.prc)*a.shrout>10000000
                      """, date_cols=['date'])

In [ ]:
crsp = crsp.loc[:,~crsp.columns.duplicated()]
crsp.loc[:, 'siccd'] /=100.0
crsp['siccd'] = crsp['siccd'].astype(int)
crsp['siccd']

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0        35
1        73
2        37
3        73
4        73
         ..
16868    48
16869    73
16870    27
16871    27
16872    79
Name: siccd, Length: 16873, dtype: int64

In [ ]:
crsp = crsp.drop(crsp[crsp.siccd > 100].index)
crsp = crsp.drop(crsp[crsp.siccd == 0].index)
crsp['siccd'].nunique()

49

In [ ]:
def getMajorGroup(row):
  val = row['siccd']
  #print("computing : " , val)
  if val >= 0 and val <=9 :
    return 1
  elif val >= 10 and val <=14 :
   return 2
  elif val >= 15 and val <=17 :
   return 3
  elif val >= 20 and val <=39 :
   return 4
  elif val >= 40 and val <=49 :
   return 5
  elif val >= 50 and val <=51 :
   return 6
  elif val >= 52 and val <=59 :
   return 7
  elif val >= 60 and val <=67 :
   return 8
  elif val >= 70 and val <=89 :
   return 9
  elif val >= 90 and val <=99 :
   return 10
  else :
    return -1

In [ ]:
df = crsp
df.reset_index(drop=True)
df.index = range(len(df))

In [ ]:
df['siccd'] = df.apply(lambda x : getMajorGroup(x), axis=1)
df['siccd']

0        4
1        9
2        4
3        9
4        9
        ..
16868    5
16869    9
16870    4
16871    4
16872    9
Name: siccd, Length: 16873, dtype: int64

In [ ]:
print(df['siccd'].value_counts())

4    6456
8    3817
5    2458
9    1557
2    1225
7    1107
6     150
3     103
Name: siccd, dtype: int64


In [ ]:
#df10 = df[df['siccd'] == 9]
df10 = df[['date', 'prc', 'ticker']]
print(df10['ticker'].value_counts())

NWS     100
BRK     100
SUNW     50
CMX      50
APOL     50
       ... 
ITT       3
CTX       3
ROK       3
LTD       2
KMG       1
Name: ticker, Length: 360, dtype: int64


In [ ]:
df10.reset_index(drop=True)
df10.index = range(len(df10))

In [ ]:
df10.groupby(['date']).mean()

,prc
date,
2005-06-17,305.996175
2005-06-20,307.782423
2005-06-21,310.245831
2005-06-22,308.290030
2005-06-23,307.284397
2005-06-24,307.447341
2005-06-27,309.730363
2005-06-28,310.281885
2005-06-29,308.374168


In [ ]:
print(type(df10.groupby(['date']).mean()))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
df_reg = df[df['siccd'] == 8]
df_reg = df_reg[['date', 'prc']]
df_reg = df_reg.groupby(['date']).mean()
extracted_col = df_temp['prc']
extracted_col.head()

0    61.730000
1    61.050001
2    64.750000
3    71.376666
4    68.965000
Name: prc, dtype: float64

In [ ]:
df_reg = df[df['siccd'] == 2]
df_reg = df_reg[['date', 'prc']]

#df_reg.reset_index(drop=True)

#df_reg.index = range(len(df_reg))

df_reg = df_reg.groupby(['date']).mean()

#df_reg = df_reg.iloc[::7, :]
print(df_reg.head())


for grp in range(3, 10):
  df_temp = df[df['siccd'] == grp]
  df_temp = df_temp[['date', 'prc', 'ticker']]
  df_temp = df_temp.groupby(['date']).mean()
  #df_temp = df_temp.iloc[::7, :]
  #df_temp.reset_index(drop=True)
  #df_temp.index = range(len(df_temp))
  extracted_col = df_temp['prc']
  #print(len(df_reg), " ", len(extracted_col) )
  df_reg.insert(0, "prc"+str(grp), extracted_col)

df_reg

                  prc
date                 
2005-06-17  58.150869
2005-06-20  58.525218
2005-06-21  57.588260
2005-06-22  58.086957
2005-06-23  58.171304


,prc9,prc8,prc7,prc6,prc5,prc4,prc3,prc
date,,,,,,,,
2005-06-17,48.893750,1205.772162,47.937272,46.459999,41.409792,48.096797,61.730000,58.150869
2005-06-20,49.152813,1210.197838,48.102727,46.863332,41.507708,47.959150,60.170002,58.525218
2005-06-21,47.548438,1223.479595,47.991364,46.873333,41.441459,47.252284,60.570000,57.588260
2005-06-22,47.546562,1214.499324,48.091364,46.886667,41.571667,47.233701,60.385002,58.086957
2005-06-23,46.924375,1230.174109,47.336818,46.750000,41.528163,47.189844,59.675001,58.171304
2005-06-24,47.828710,1211.823108,46.877273,46.583333,41.310408,46.916693,58.500000,58.121305
2005-06-27,48.070323,1221.579460,46.874091,44.856667,41.342857,46.985827,59.799999,58.925652
2005-06-28,48.415806,1216.648784,47.939545,45.513332,41.768571,46.818480,61.050001,58.154783
2005-06-29,47.064687,1207.901307,47.572727,45.913333,41.488400,46.609762,60.339998,58.159130


In [ ]:
df_reg.reset_index(drop=True)
df_reg.index = range(len(df_reg))
#df_reg = df_reg.drop('date', 1)
df_reg.columns

Index(['prc9', 'prc8', 'prc7', 'prc6', 'prc5', 'prc4', 'prc3', 'prc'], dtype='object')

In [ ]:
df_reg = df_reg.dropna()
print(len(df_reg))

50


In [ ]:
#normalized_df=(df_reg-df_reg.mean())/df_reg.std()
#normalized_df=(df_reg-df_reg.min())/(df_reg.max()-df_reg.min())

# To find the discrete difference
normalized_df = df_reg.diff(axis = 0, periods = 1)


In [ ]:
normalized_df = normalized_df.iloc[1:]
normalized_df

,prc,prc2,prc3,prc4,prc5,prc6,prc7,prc8,prc9
1,-0.529999,89.810005,1.280001,-32.090004,-8.070000,-131.699997,23.540005,47.159996,89.930004
2,0.680000,-114.670006,-0.110001,55.849998,29.600000,55.379990,-3.780006,-104.149998,-25.430008
3,2.680000,-52.210002,1.180000,-26.079994,-41.580001,-86.969994,-1.059998,23.980000,-86.649998
4,-0.030003,77.700000,1.139999,-3.610001,34.889998,87.590004,-9.680000,20.270000,65.740002
5,0.380001,-30.020000,0.180000,0.329994,-12.070000,-10.050003,4.270004,-15.689999,667.829994
...,...,...,...,...,...,...,...,...,...
263,0.130001,-64.740000,21.070000,-56.590004,-26.299999,-52.360001,0.279999,-62.160001,26.020004
264,-3.150002,0.030001,-1.930000,16.820000,17.950001,-31.769997,0.529999,27.590001,70.490005
265,3.299999,1.619999,-18.490002,-35.039997,-17.280003,31.889999,-13.330002,197.290001,-92.120010
266,-2.139999,88.770000,3549.260059,48.670002,-8.730000,-49.209999,636.160004,-202.170002,15.360001


In [ ]:
from sklearn import linear_model

X = df_reg[['prc3', 'prc4', 'prc5', 'prc6', 'prc7', 'prc8', 'prc9']]
y = df_reg['prc']


In [ ]:
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# importing module
from sklearn.linear_model import LinearRegression
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x_train,y_train)

print('Intercept:', LR.intercept_)
print('Coefficients:', LR.coef_)

Intercept: 1.4354967303321544
Coefficients: [-0.03731832  1.49224783  0.83238981 -0.48750605 -0.45996289  0.00533067
 -0.14957323]


In [ ]:
y_prediction =  LR.predict(x_test)

In [ ]:
# importing r2_score module
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print("r2 socre is ",score)
print("mean_sqrd_error is==",mean_squared_error(y_test,y_prediction))
print("root_mean_squared error of is==",np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 socre is  0.22588871483694395
mean_sqrd_error is== 1.68705802307733
root_mean_squared error of is== 1.298867977539415


In [ ]:
# Fitting Polynomial Regression to the dataset
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=2)
X_poly = poly_reg.fit_transform(x_train)

clf = linear_model.LinearRegression()
train_y_ = clf.fit(X_poly, y_train)
# The coefficients
#print ('Coefficients: ', clf.coef_)
print ('Intercept: ',clf.intercept_)

Intercept:  75.35329190544704


In [ ]:
from sklearn.metrics import r2_score

test_x_poly = poly_reg.fit_transform(x_test)
test_y_ = clf.predict(test_x_poly)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - y_test) ** 2))
print("R2-score: %.2f" % r2_score(test_y_ , y_test) )

Mean absolute error: 34.16
Residual sum of squares (MSE): 3535.74
R2-score: -0.72
